In [6]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import itertools
from utils.data_loader import DataLoader

In [7]:
loader = DataLoader('../data/data_speeches.csv', '../data/data_statements.csv')
loader.process()

docs = loader.get_docs()
sentences = loader.get_sentences()

embeddings_gr = loader.load_embeddings("../data/embeddings/embeddings_gr.pkl")
embeddings_gr_media = loader.load_embeddings("../data/embeddings/embeddings_gr_media.pkl")
embeddings_gr_sentences = loader.load_embeddings("../data/embeddings/embeddings_gr_sentences.pkl")
embeddings_gr_media_sentences = loader.load_embeddings("../data/embeddings/embeddings_gr_media_sentences.pkl")

In [8]:
st_greek = SentenceTransformer('lighteternal/stsb-xlm-r-greek-transfer')
st_greek_media = SentenceTransformer('dimitriz/st-greek-media-bert-base-uncased')

In [9]:
print(f"Number of all docs: {len(docs)}")
print(f"Number of all sentences: {len(sentences)}")

Number of all docs: 2012
Number of all sentences: 212471


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
import advertools as adv

In [18]:
stopwords = adv.stopwords['greek']
stopwords = list(stopwords) +  ['χώρα', 'ελλάδα', 'κύριοι', 'κυρία', 'κυρίες', 'κύριο', 'συμπολίτες']
vectorizer_model = CountVectorizer(stop_words=stopwords, min_df=2, max_df=8, ngram_range=(1, 2))

In [41]:
topic_model = BERTopic(embedding_model=st_greek_media, vectorizer_model=vectorizer_model)
topics, _ = topic_model.fit_transform(docs, embeddings_gr_media)

In [42]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,725,-1_the_and_of_to,"[the, and, of, to, σήμερα, ευρώπη, χρόνια, in,...",[«Στόχος μας σε ένα χρόνο από τώρα η χώρα να β...
1,0,245,0_κι_σήμερα_χρόνια_κυβέρνηση,"[κι, σήμερα, χρόνια, κυβέρνηση, χώρας, νέα, δι...",[ΓΡΑΦΕΙΟ ΤΥΠΟΥ ΠΡΩΘΥΠΟΥΡΓΟΥ Πέμπτη 30 Ιανουαρί...
2,1,113,1_υγείας_σήμερα_σύστημα_σύστημα υγείας,"[υγείας, σήμερα, σύστημα, σύστημα υγείας, νοσο...",[Φίλες και φίλοι είναι η δεύτερη φορά που επισ...
3,2,62,2_ευρωπαϊκής_ευρωπαϊκή_χώρες_ένωσης,"[ευρωπαϊκής, ευρωπαϊκή, χώρες, ένωσης, σήμερα,...",[Κύριοι Πρόεδροι κυρίες και κύριοι Ολοκληρώσαμ...
4,3,61,3_προστασίας_πολιτικής προστασίας_θέλω_κάνουμε,"[προστασίας, πολιτικής προστασίας, θέλω, κάνου...",[Δεν νομίζω ότι χρειάστηκε κα Κωνσταντοπούλου ...
5,4,55,4_ευρώ_αγρότες_τομέα_προϊόντα,"[ευρώ, αγρότες, τομέα, προϊόντα, θέλω, σήμερα,...",[Γεια σας! Σας ευχαριστώ θερμά γι αυτή την υπο...
6,5,53,5_τουρκία_χώρες_ροές_ασύλου,"[τουρκία, χώρες, ροές, ασύλου, σύνορα, ευρωπαϊ...",[Οι κοινές δηλώσεις του Πρωθυπουργού Κυριάκου ...
7,6,50,6_ευρώπη_σήμερα_βίας_γυναίκες,"[ευρώπη, σήμερα, βίας, γυναίκες, γυναικών, χώρ...",[Κυρίες και κύριοι Είναι πολύ σημαντικό ότι η ...
8,7,49,7_the_and_to_of,"[the, and, to, of, is, in, we, that, it, for]",[ΓΡΑΦΕΙΟ ΤΥΠΟΥ ΠΡΩΘΥΠΟΥΡΓΟΥ Τετάρτη 2 Οκτωβρίο...
9,8,45,8_ευρώπη_γερμανία_πιστεύω_ζήτημα,"[ευρώπη, γερμανία, πιστεύω, ζήτημα, θέλω, σήμε...",[Κυριάκος Μητσοτάκης: Αγαπητέ Καγκελάριε της Ο...


In [19]:
topic_model = BERTopic(embedding_model=st_greek, vectorizer_model=vectorizer_model, nr_topics='auto')
topics, _ = topic_model.fit_transform(docs, embeddings_gr)

In [20]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,672,-1_they_as_but_democracy,"[they, as, but, democracy, if, their, was, ref...",[ΓΡΑΦΕΙΟ ΤΥΠΟΥ ΠΡΩΘΥΠΟΥΡΓΟΥ Τετάρτη 4 Δεκεμβρί...
1,0,166,0_εμβολίων_εμβολιασμός_εμβολιασμούς_κλίνες,"[εμβολίων, εμβολιασμός, εμβολιασμούς, κλίνες, ...",[«Αυτές οι μονάδες τις οποίες εγκαινιάσαμε είν...
2,1,124,1_μομφής_πρόταση μομφής_κατηγορείτε_εκατόν,"[μομφής, πρόταση μομφής, κατηγορείτε, εκατόν, ...",[Ακολουθούν παρακάτω η δευτερολογία και η τριτ...
3,2,76,2_θεόφιλος_τάφου_πατριάρχης_ιεροσολύμων,"[θεόφιλος, τάφου, πατριάρχης, ιεροσολύμων, εξέ...",[ΓΡΑΦΕΙΟ ΤΥΠΟΥ ΠΡΩΘΥΠΟΥΡΓΟΥ Παρασκευή 11 Οκτωβ...
4,3,74,3_ταυτοποίησης_προσφυγικών ροών_προσφύγων μετα...,"[ταυτοποίησης, προσφυγικών ροών, προσφύγων μετ...",[ΓΡΑΦΕΙΟ ΤΥΠΟΥ ΠΡΩΘΥΠΟΥΡΓΟΥ Παρασκευή 7 Ιουνίο...
5,4,62,4_αστυπάλαιας_τιμές ηλεκτρικής_ηλεκτρικό_ηλεκτ...,"[αστυπάλαιας, τιμές ηλεκτρικής, ηλεκτρικό, ηλε...",[Κυρίες και κύριοι βουλευτές κλείνει σχεδόν έν...
6,5,61,5_οικείους_θερμά συλλυπητήρια_αποχαιρετούμε_τι...,"[οικείους, θερμά συλλυπητήρια, αποχαιρετούμε, ...",[ΓΡΑΦΕΙΟ ΤΥΠΟΥ ΠΡΩΘΥΠΟΥΡΓΟΥ Τετάρτη 6 Μαρτίου ...
7,6,46,6_ε65_οακα_βοακ_μεταφορών κώστας,"[ε65, οακα, βοακ, μεταφορών κώστας, οδικό άξον...",[Στα εγκαίνια της Παράκαμψης Αμφιλοχίας που ολ...
8,7,41,7_microsoft_cloud_google_brad,"[microsoft, cloud, google, brad, smith, viva, ...",[Ο Πρωθυπουργός Κυριάκος Μητσοτάκης συμμετείχε...
9,8,37,8_δάσος_εναέρια_δαδιά_ελγα,"[δάσος, εναέρια, δαδιά, ελγα, δάσους, εκκενώσε...",[Ευχαριστώ κ. Πρόεδρε. Επιτρέψτε μου να ξεκινή...


In [46]:
topic_model = BERTopic(embedding_model=st_greek, vectorizer_model=vectorizer_model)
topics, _ = topic_model.fit_transform(sentences, embeddings_gr_sentences)

KeyboardInterrupt: 